In [1]:
import requests
import re
import shopify
from bs4 import BeautifulSoup
from collections import Counter
import logging
from dotenv import load_dotenv
import os

logging.basicConfig(level=logging.ERROR)

shop_url = "https://429eef-90.myshopify.com/"
api_version = '2024-01'
# state = binascii.b2a_hex(os.urandom(15)).decode("utf-8")
# redirect_uri = "http://myapp.com/auth/shopify/callback"
# scopes = ['read_apps', 'write_files', 'read_files', 'write_products', 'read_products', 'read_content', 'write_content', 'write_product_feeds', 'read_product_feeds', 'write_product_listings', 'read_product_listings']

load_dotenv()
access_token = (os.getenv('access_token'))

session = shopify.Session(shop_url, api_version, access_token)
shopify.ShopifyResource.activate_session(session)


In [2]:
class Product:
    def __init__(self,handle, name, spu, imgs, url, detail=None):
        self.name = name
        self.handle = handle
        self.detail = """

        <div class="value"><p><span style="font-weight: 400;">Gel Polish is a new generation of nails care!</span></p>
<p><span style="font-weight: 400;">Forget about not lasting and peeling off colors, experience super shiny lasting and incredible quality of ReformA Gel Polishes.</span></p>
<p><span style="font-weight: 400;">Nine reasons thanks to which you will love them:</span></p>
<ol>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They have beautiful colors, you can find juicy neons as well as delicate nudes and all shades between them - a total of over 300 and we still introduce new ones!</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">Even a beginner could apply them easily - they have consistence of yoghurt so you can spread them comfortably.</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They have delicate smell.</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They cure fast so you save time and protect nails.</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They are self-levelling and have good adhesion to natural nails, gel, acrylic and acrylic gel.</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They are ideal for manicure and pedicure.</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They are incredibly lasting.</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They don't peel or flake off.</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They guarantee beautiful covering after applying 1-2 layers.</span></li>
</ol>
<p><span style="font-weight: 400;"> </span></p>
<p><span style="font-weight: 400;">All gel polishes are available in bottles with brushes of capacity 10ml, and some of them also in bottles of capacity 3ml.</span></p>
<p><span style="font-weight: 400;">Try also our incredible bases and tops.</span></p>
<p><strong>ReformA Educator's advices:</strong></p>
<p><span style="font-weight: 400;">When you apply gel polish remember not to touch the underside with a brush.</span></p>
<p><span style="font-weight: 400;">Delicately stroking with a brush, apply gel polish in the air - then you will get smooth and beautiful surface.</span></p>
<p><span style="font-weight: 400;">Each color apply on cured base and at the end cover it with top.</span></p>
<p><span style="font-weight: 400;">Curing time in a UV lamp: 2 minutes, in an LED lamp: 1min - 20sec, in a 48W Dual LED lamp: 30 - 20sec.</span></p>
<p><span style="font-weight: 400;">The curing time of gel products and gel polishes depends on the power of lamp and bulbs, always check individually the curing time of gel or gel polish.</span></p>
"""
        if detail:
            self.detail = str(detail)
        self.spu = spu
        self.imgs = imgs
        self.url = url
        self.price = '10.00'
        self.compare_at_price = '13.00'
        self.grams = 15
        self.weight = 0.015
        self.vendor = 'Reforma'
        self.product_type = 'Gel Polish'

    def __str__(self):
        return f"Product: {self.name}, SPU: {self.spu}"


In [3]:
def extract_src_original(script_content):
  """
  Extracts "SRC_ORIGINAL" values from the provided Javascript snippet containing RSGoPro_Pictures data and prepends base URL.

  Args:
      script_content (str): The Javascript code snippet containing image data.

  Returns:
      list: A list of full image URLs with the base URL prepended, or an empty list if no matches found.
  """
  base_url = 'https://reformacosmetics.com'
  src_original_matches = re.findall(r"'SRC_ORIGINAL':'([^']+)'", str(script_content))
  return [f"{base_url}{match}" for match in src_original_matches] or []


In [4]:
def get_search_results(url):
    """
    Получает результаты поиска для заданного URL.
    """
    try:
        with requests.get(url) as response:
            if response.status_code == 200:
                return response.text
            else:
                logging.error("Ошибка при получении страницы с результатами поиска: %d", response.status_code)
                return None
    except requests.RequestException as e:
        logging.error("Ошибка при запросе страницы: %s", str(e))
        return None


In [5]:
def extract_product_links(url):
    """
    Извлекает ссылки на продукты из HTML-кода страницы с результатами поиска.
    """
    base_url = 'https://reformacosmetics.com'

    html = get_search_results(url)
    if html:
        soup = BeautifulSoup(html, 'html.parser')
        divs = soup.find_all('div', class_='list-showcase__name')
        href_links = [div.a['href'] for div in divs if div.a]
        return [f"{base_url}{match}" for match in href_links] or []


In [6]:
def letsgo(page):
    print()

    # url = 'https://reforma.top/catalogsearch/result/?q=' + SPU + '&product_list_limit=144'
    url = 'https://reformacosmetics.com/catalog/nogti/gel-polishes/?PAGEN_1='+str(page)
    print(url)
    product_links = extract_product_links(url)
    if product_links:
        print("List created")
        return product_links
        

In [7]:
def findEANsku(sku):

    print(sku)

    url = 'https://www.tufishop.de/en/?subcats=Y&pcode_from_q=Y&pname=Y&search_performed=Y&q='+str(sku)+'&dispatch=products.search&_gl=1*1o43pxb*_up*MQ..*_ga*MTY3MTAzNzE3Mi4xNzEzODEyNzUw*_ga_Q8841MRH6K*MTcxMzgxMjc0OS4xLjAuMTcxMzgxMjc0OS4wLjAuMA..'

    html = get_search_results(url)
    if html:
        soup = BeautifulSoup(html, 'html.parser')
        links = soup.find_all('a', class_='product-title')
        if not links:
            print('no link:'+sku)
            print()
        else:
            link = links[0]['href']
            html = get_search_results(link)
            if html:
                soup = BeautifulSoup(html, 'html.parser')
                ean_label = soup.find('label', text='EAN:')

                if ean_label:
                    ean = ean_label.find_next_sibling('span').get_text(strip=True)

                    print(ean)
                    return ean


In [8]:
def create_product(product):
    """
    Creates a new product on Shopify with the provided data.
    """
    try:
        print('Start  creat product.')

        Handle = product.handle
        product_shopify = shopify.Product.find(handle=Handle)
        if not product_shopify:
            # Create a new product instance
            product_shopify = shopify.Product()

            # Set basic product details
            product_shopify.title = product.name
            product_shopify.handle = Handle
            product_shopify.body_html = product.detail
            product_shopify.vendor = product.vendor
            product_shopify.product_type = product.product_type

            # Create a default variant
            default_variant = {
                'title': 'Default Title',
                'price': product.price,
                'sku': product.spu,
                'compare_at_price': product.compare_at_price,
                'inventory_policy': 'deny',
                'fulfillment_service': 'manual',
                'inventory_management': 'shopify',
                'option1': 'Default Title',
                'taxable': True,
                'barcode': findEANsku(product.spu),
                'grams': product.grams,
                'weight': product.weight,
                'weight_unit': 'kg',
                'requires_shipping': True
            }

            # Create a Variant object for the default variant
            variant = shopify.Variant(default_variant)

            # Add the default variant to the product
            product_shopify.variants = [variant]

            # Add images to the product
            images = [{'src': img_url, 'position': index, 'alt': product.name} for index, img_url in enumerate(product.imgs, start=1)]
            product_shopify.images = images
            logging.info("Product created successfully!")
            print("Product created successfully!")

            product_shopify.save()
            
            print('Product save successfully!')

        else:
            # Update the product title
            # product_shopify = product_shopify[0]

            
            # product_shopify.handle = Handle

            logging.info("Product modified successfully!")
            print("Product already exist!")

        # variants = product_shopify.variants
        # variant = variants[0]
        # print(variant)
        
        # if not variant.barcode:
        
        #     print(variant.sku,"need")
        # else :
        #     print("Product barcode exist!")
        
        # Save the product
        # product_shopify.save()

        return product_shopify
    except Exception as e:
        logging.error(f"Failed to create product: {str(e)}")
        return None

In [9]:
def extract_data_from_url(url):
    """
    Extracts product data from the provided URL.
    """
    try:
        print('Start Data extracted.')

        # Extract data from the URL
        html = get_search_results(url)
        # print(html)
        if not html:
            raise ValueError("Failed to fetch HTML content from the URL.")

        handle = url.split('/')[-2].replace('eol-','') .rstrip('r')
        soup = BeautifulSoup(html, 'html.parser')       
        spu = soup.find('span', class_='c-article__value js-article__value').text
        title = ''.join([' ' + part.capitalize() + (' -' if part == 'polish' or part == 'base' else '') if part[0].isalpha() else ', ' + part for part in handle.split('-')]).replace(' -,',',')
        detail = None
        imgs = []
        script_data = soup.find('script', string=lambda text: text and soup.find('h1').text in text)
        # print(soup)
        # print(script_data)
        if script_data:
            imgs = extract_src_original(script_data)

        if 'nabor' in handle:
            detail = soup.find('div', class_='tab-pane-in2')
        
        elif handle[-1] == 'g':
            handle = handle[:-2] + handle[-1:] 
        else:
            handle = handle[:-3] + handle[-2:]

        print('Data extracted.')
        print(title)
        return Product(handle, title, spu, imgs, url,detail)
    except Exception as e:
        logging.error(f"Failed to extract data from URL: {str(e)}")
        return None

In [10]:
def UkRefSearch(sku):
    print('Searching for '+str(sku))
    url = 'https://reformacosmetics.com/search/?q='+str(sku)

    html = get_search_results(url)
    if html:
        soup = BeautifulSoup(html, 'html.parser')
        links = soup.find_all('a', class_='nm')
        if links:
            return 'https://reformacosmetics.com'+links[0]['href']
        return None
    return None


In [18]:
for i in range(1,100):
    create_product(extract_data_from_url(UkRefSearch(input('sku'))))

Searching for 941105
Start Data extracted.
Data extracted.
 Gel Polish - Top Model, 10 Ml
Start  creat product.
Product already exist!
Searching for 941889
Start Data extracted.
Data extracted.
 Gel Polish - Pi Acolada, 10 Ml
Start  creat product.
941889


/tmp/ipykernel_24261/1993197726.py:19: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  ean_label = soup.find('label', text='EAN:')


5055379418890
Product created successfully!
Product save successfully!
Searching for 941956
Start Data extracted.
Data extracted.
 Gel Polish, 4ever, 10 Ml
Start  creat product.
941956
5055379419569
Product created successfully!
Product save successfully!
Searching for 941742
Start Data extracted.
Data extracted.
 Gel Polish - Nike, 10 Ml
Start  creat product.
941742
5055379417428
Product created successfully!
Product save successfully!
Searching for 941109
Start Data extracted.
Data extracted.
 Gel Polish - Harlem, 10 Ml
Start  creat product.
Product already exist!
Searching for 941853
Start Data extracted.
Data extracted.
 Gel Polish - Tears Of Rain, 10 Ml
Start  creat product.
941853
5055379418531
Product created successfully!
Product save successfully!
Searching for 941105
Start Data extracted.
Data extracted.
 Gel Polish - Top Model, 10 Ml
Start  creat product.
Product already exist!
Searching for 941486
Start Data extracted.
Data extracted.
 Gel Polish - Las Vegas, 10 Ml
Start  c

ERROR:root:Ошибка при запросе страницы: Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?
ERROR:root:Failed to extract data from URL: Failed to fetch HTML content from the URL.
ERROR:root:Failed to create product: 'NoneType' object has no attribute 'handle'


Start Data extracted.
Start  creat product.


KeyboardInterrupt: Interrupted by user

In [12]:
data = extract_data_from_url('https://reformacosmetics.com/product/nabor-fresh-air/')
print(data.detail
      )

Start Data extracted.
Data extracted.
 Nabor Fresh Ai
<div class="tab-pane-in2">В набір входять:<br>
Concrete 942029<br>
Baby blue 941894<br>
Wild orchid 941189<br>
Love affair 941209<br/>
Soft touch 941213<br/>
Hugs 941178<br/>
Frosting 942027<br/>
</br></br></br></br></div>


In [13]:
html = get_search_results('https://reformacosmetics.com/product/gel-polish-pink-pearl-3-ml/')
soup = BeautifulSoup(html, 'html.parser')       
print(soup)

<!DOCTYPE html>

<html itemscope="" itemtype="http://schema.org/WebSite" lang="ru" xml:lang="ru">
<head>
<title>Гель-лак Pink Pearl, 3мл - описание, цены - Реформа</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Гель-лак Pink Pearl, 3мл" name="keywords"/>
<meta content="Описание, цены Гель-лак Pink Pearl, 3мл - Реформа" name="description"/>
<script data-skip-moving="true" type="text/javascript">(function(w, d, n) {var cl = "bx-core";var ht = d.documentElement;var htc = ht ? ht.className : undefined;if (htc === undefined || htc.indexOf(cl) !== -1){return;}var ua = n.userAgent;if (/(iPad;)|(iPhone;)/i.test(ua)){cl += " bx-ios";}else if (/Android/i.test(ua)){cl += " bx-android";}cl += (/(ipad|iphone|android|mobile|touch)/i.test(ua) ? " bx-touch" : " bx-no-touch");cl += w.devicePixelRatio && w.devicePixelRatio >= 2? " bx-retina": " bx-no-retina";var ieVersion = -1;if (/AppleWebKit/.test(ua)){cl += " bx-chrome";}else if ((ieVersion = getIeVersion(

In [14]:
prod_link = 'https://reformacosmetics.com/product/gel-polish-pink-pearl-3-ml/'
print(prod_link)
create_product(extract_data_from_url(prod_link))

https://reformacosmetics.com/product/gel-polish-pink-pearl-3-ml/
Start Data extracted.
Data extracted.
 Gel Polish - Pink Pearl, 3 Ml
Start  creat product.
Product already exist!


[product(8773956272458)]

In [15]:
len(extract_product_links('https://reformacosmetics.com/catalog/nogti/gel-polishes/?PAGEN_1=1'))

15

In [16]:
# for iindex in range(1, 5):
#     prod_links = letsgo(iindex)
#     for index, prod_link in enumerate(prod_links, start=1):
#         print(prod_link)
#         create_product(extract_data_from_url(prod_link))
#         print(f"{index}/{len(prod_links)}.{iindex}\n")

In [17]:
UkRefSearch(941649)


Searching for 941649


'https://reformacosmetics.com/product/eol-gel-polish-crocuses-10-ml/'